In [ ]:
!pip install pandas
!pip install cx_oracle
!pip install tourch
!pip install transformers

In [1]:
import pandas as pd
import cx_Oracle
from tabulate import tabulate
from transformers import pipeline
import xml.etree.ElementTree as ET
from io import StringIO

In [11]:

import oracledb
import random
import re
import webbrowser
from lxml import etree
# #SCRE
# username = 'svcGOCUI'
# password = 'DELL2023support#'
# host = 'gocplorlvpr18.amer.dell.com'
# port = '1521'
# service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'

#GE2-CIS
dbname="GE2-CIS"
username = 'MWMGR'
password = 'Saynotoclown$$$1'
host = 'ausul2trsdb02.us.dell.com'
port = '1521'
service_name = 'wwg2s.sit.amer.dell.com'
excel_name="DataFromDB_"+dbname+".xlsx"


def retrieve_data():
    dsn = oracledb.makedsn(host, port, service_name=service_name)
    connection = None
    
    try:
        connection = oracledb.connect(user=username, password=password, dsn=dsn)
        print("Connected to Oracle Database")
        
        cursor = connection.cursor()
        query = "SELECT original_msg FROM transaction_log WHERE message_type = 'CIS_OUTBOUND_OMEGA' AND business_lookup_key = '000000000'"
        
        cursor.execute(query)
        rows = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]

        data = pd.DataFrame(rows, columns=column_names)
        
        for col in data.select_dtypes(include=['object']).columns:
            print('col',data[col])
            data[col] = data[col].apply(lambda x: x.read() if hasattr(x, 'read') else x)
        
        # data.to_excel('DataFromDB.xlsx', index=False)
        return data
    
    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        if connection:
            connection.close()
            print("Connection closed")


In [12]:
data = retrieve_data()
data.head()

Connected to Oracle Database
    REGION       FORECAST_DATE  MODEL1  MODEL2  MODEL3  MODEL4 MODEL5  \
0      APJ 2024-07-26 05:00:00     915     924     876     396   None   
1      APJ 2024-07-26 06:00:00     985    1008    1048     388   None   
2      APJ 2024-07-26 10:00:00    1016     999     820     282   None   
3     EMEA 2024-07-26 10:00:00     953     949     982     310   None   
4     EMEA 2024-07-28 21:00:00     610     554     125     103   None   
..     ...                 ...     ...     ...     ...     ...    ...   
493   EMEA 2024-07-28 19:00:00      51      72      99     162   None   
494    DAO 2024-07-28 19:00:00     427     480     471     849   None   
495    APJ 2024-07-28 19:00:00       4       7      42      18   None   
496   EMEA 2024-07-28 20:00:00     218     197     110     131   None   
497    APJ 2024-07-28 21:00:00       9      66      51      28   None   

     ACTUALORDERUNIT CREATED_DATE UPDATED_DATE  
0              850.0   2024-07-25   2024-07-2

,REGION,FORECAST_DATE,MODEL1,MODEL2,MODEL3,MODEL4,MODEL5,ACTUALORDERUNIT,CREATED_DATE,UPDATED_DATE
0,APJ,2024-07-26 05:00:00,915,924,876,396,None,850.0,2024-07-25,2024-07-25
1,APJ,2024-07-26 06:00:00,985,1008,1048,388,None,852.0,2024-07-25,2024-07-25
2,APJ,2024-07-26 10:00:00,1016,999,820,282,None,756.0,2024-07-25,2024-07-25
3,EMEA,2024-07-26 10:00:00,953,949,982,310,None,951.0,2024-07-25,2024-07-25
4,EMEA,2024-07-28 21:00:00,610,554,125,103,None,95.0,2024-07-25,2024-07-25


In [5]:
xml_str=data['ORIGINAL_MSG'][0]

xml_str = xml_str.strip()
print(xml_str)

<?xml version="1.0" encoding="UTF-8"?>
<Response>
   <orderNumber>000000000</orderNumber>
   <buid>3535</buid>
   <organizationId>3535</organizationId>
   <result>OPEN</result>
   <createdDate>2022-03-31T13:25:51</createdDate>
   <screenDate>2022-07-08T04:46:12</screenDate>
   <ReasonCode></ReasonCode>
</Response>


In [6]:
                                    #Method to extract CIS_OUTBOUND_OMEGA XML Tags
from bs4 import BeautifulSoup
def Extract_CIS_OUTBOUND_OMEGA_Tag(tag):
    bs = BeautifulSoup(xml_str, 'xml')
    res = bs.find(tag)
    if res:
        # print(res)
        res_content = res.get_text().strip()
        # print(res_content)
        return res_content 
    else:
        print("tag not found")
tag_name='result'
msg = Extract_CIS_OUTBOUND_OMEGA_Tag(tag_name)
print(msg)

OPEN


In [ ]:
                            #Method to extract OMS_INBOUND XML Tags
def get_namespaces(xml_str):
    events = "start", "start-ns"
    ns_map = []
    
    f = StringIO(xml_str)
    
    for event, elem in ET.iterparse(f, events):
        if event == "start-ns":
            ns_map.append(elem)
        elif event == "start":
            root = elem
            break

    return root, dict(ns_map)

root, namespace = get_namespaces(xml_str)

# Extract information from the XML using dynamically retrieved namespaces
for person in root.findall(".//{{{}}}partners/{{{}}}partner".format(namespace['v1'], namespace['v1'])):
    orderNumber = person.find("{{{}}}orderNumber".format(namespace['v1'])).text
    state_code = person.find("{{{}}}stateCode".format(ns['v1'])).text
    print(f"orderNumber: {orderNumber}, State Code: {state_code}")


<Element 'Response' at 0x000001D646F825C0>
{}


In [ ]:
import xml.etree.ElementTree as ET

# Function to extract namespaces
def get_namespaces(xml_str):
    events = "start", "start-ns"
    root = None
    ns_map = []

    for event, elem in ET.iterparse(xml_str, events):
        if event == "start-ns":
            ns_map.append(elem)
        elif event == "start":
            root = elem
            break

    return root, dict(ns_map)

# Get the root and namespaces
root, ns = get_namespaces(xml_str)

# Extract information from the XML using dynamically retrieved namespaces
for person in root.findall(".//{{{}}}partners/{{{}}}partner".format(ns['v1'], ns['v1'])):
    name = person.find("{{{}}}name".format(ns['v1'])).text
    state_code = person.find("{{{}}}stateCode".format(ns['v1'])).text
    print(f"Name: {name}, State Code: {state_code}")


In [ ]:
 # Extract <v1:partner> tag from XML
def extract_partner(xml_str):
    try:
        root = ET.fromstring(xml_str)
        namespaces = {'v1': root.tag.split('}')[0].strip('{')}
        partner = root.find('.//v1:partner', namespaces)
        return partner.text if partner is not None else None
    except ET.ParseError as e:
                print(f"Error parsing XML: {e}")
                return None
data = data.apply(extract_partner)

In [50]:
import xml.etree.ElementTree as ET
tree = ET.ElementTree(ET.fromstring(xml_str))
type(tree)
for ele in tree.iter():
    print(ele.)


In [ ]:
# Parse the XML string
tree = ET.ElementTree(ET.fromstring(xml_str))

# Iterate over all elements and find the tag by its local name
partner = None
for elem in tree.iter():
    print(elem)
    if elem.tag.endswith('<v1:partners>'):
        partner = elem
        break

# Check if the tag was found and print its content
if partner is not None:
    print(ET.tostring(partner, encoding='unicode'))
else:
    print("Tag <partner> not found.")

In [43]:
from xml.etree.ElementTree import XML, fromstring
myxml = fromstring(xml_str)
print(myxml)

<Element '{http://schemas.dell.com/dbi/ts/compliance/v1_0}Compliance' at 0x000001D621162C00>

In [ ]:
import xml.etree.ElementTree as ET
element = ET.XML(myxml)
ET.indent(element)
print(ET.tostring(element, encoding='unicode'))

In [19]:
# Oracle database connection details
username = 'svcGOCUI'
password = 'DELL2023support#'
host = 'gocplorlvpr18.amer.dell.com'
port = '1521'
service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'
# Oracle database connection details
# username = 'SERVICE_EMEA_EFDR_SAS_SUPT'
# password = 'dmGM29JLiFjz5JBwDDx8#'
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)

def connect_to_oracle():
    try:
        connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)
        print("Connected to Oracle Database")
        return connection
    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Oracle error code: {error.code}")
        print(f"Oracle error message: {error.message}")
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [11]:
#running query 
def execute_query(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]

        if rows:
            table = tabulate(rows, headers=column_names, tablefmt="pretty")
            print(table)
        else:
            print("No results found.")
    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Oracle error code: {error.code}")
        print(f"Oracle error message: {error.message}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cursor.close()

In [ ]:
def is_sql_query(user_input):
    # Basic check for common SQL keywords
    sql_keywords = ["SELECT", "INSERT", "UPDATE", "DELETE", "FROM", "WHERE", "JOIN", "CREATE", "DROP", "ALTER"]
    return any(keyword in user_input.upper() for keyword in sql_keywords)

def main():
    connection = connect_to_oracle()
    if not connection:
        return
    
    # Load the transformer model
    nlp = pipeline("question-answering")

    try:
        while True:
            user_input = input("You: ").strip()
            if user_input.lower() == 'exit':
                break

            if is_sql_query(user_input):
                execute_query(connection, user_input)
            else:
                context = "Your context about the database or anything else"
                result = nlp(question=user_input, context=context)
                print(f"Bot: {result['answer']}")
    finally:
        if connection:
            connection.close()
            print("Connection closed")

if __name__ == "__main__":
    main()

In [128]:
import pandas as pd
from sqlalchemy import create_engine
import cx_Oracle

In [135]:
#SCRE
username = 'svcGOCUI'
password = 'DELL2023support#'
host = 'gocplorlvpr18.amer.dell.com'
port = '1521'
service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'

In [ ]:
dsn = oracledb.makedsn(host, port, service_name=service_name)
try:
    connection = None
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connected to Oracle Database")

    data = pd.read_csv("Future_forecasted_Copy.csv")
    # print(data)
    print(type(data))
    data.to_sql('tb_orderprocess_effectiveness_forecast', con=connection, if_exists='replace', index=False)
    
except Exception as e:
    print(f"An error occurred: {e}")
    
finally:
    if connection:
        connection.close()
        print("Connection closed")

In [143]:
!pip install cx_oracle
!pip install sqlalchemy  
!pip install cx_Oracle

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [159]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, String, DateTime, Numeric
import cx_Oracle

#SCRE
username = 'svcGOCUI'
password = 'DELL2023support#'
host = 'gocplorlvpr18.amer.dell.com'
port = '1521'
service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'
dsn = oracledb.makedsn(host, port, service_name=service_name)

engine = create_engine(f'oracle+cx_oracle://{username}:{password}@{dsn}')
dtype = {
    'REGION': String(256), 
    'FORECAST_DATE': DateTime,
    'MODEL1': Numeric,
    'MODEL2': Numeric,
    'MODEL3': Numeric,
    'MODEL4': Numeric,
    'MODEL5': Numeric,
    'ACTUALORDERUNIT': Numeric,
    'CREATED_DATE': DateTime,
    'UPDATED_DATE': DateTime,
}

try:
    data = pd.read_csv("Future_forecasted_OIF_SQL.csv")
    print(type(data))
    data['FORECAST_DATE'] = pd.to_datetime(data['FORECAST_DATE'])
    data['CREATED_DATE'] = pd.to_datetime(data['CREATED_DATE'])
    data['UPDATED_DATE'] = pd.to_datetime(data['UPDATED_DATE'])
    
    print(type(data))
    data.to_sql('tb_orderprocess_effectiveness_forecast',schema ='work', con=engine, if_exists='append', index=False)
    print("added successfully")
    
except Exception as e:
    print(f"An error occurred: {e}")
    
finally:
    engine.dispose()
    print("Connection closed")


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_21420\2519587116.py:28: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['FORECAST_DATE'] = pd.to_datetime(data['FORECAST_DATE'])
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_21420\2519587116.py:29: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['CREATED_DATE'] = pd.to_datetime(data['CREATED_DATE'])
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_21420\2519587116.py:30: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['UPDATED_DATE'] = pd.to_datetime(data['UPDATED_DATE'])


Data inserted successfully
Connection closed


In [20]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import String, DateTime, Numeric
import cx_Oracle

#SCRE
username = 'svcGOCUI'
password = 'DELL2023support#'
host = 'gocplorlvpr18.amer.dell.com'
port = '1521'
service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'

# Create DSN
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)

# Create engine
engine = create_engine(f'oracle+cx_oracle://{username}:{password}@{dsn}')

# Data types for the table
dtype = {
    'REGION': String(256), 
    'FORECAST_DATE': DateTime,
    'MODEL1': Numeric,
    'MODEL2': Numeric,
    'MODEL3': Numeric,
    'MODEL4': Numeric,
    'MODEL5': Numeric,
    'ACTUALORDERUNIT': Numeric,
    'CREATED_DATE': DateTime,
    'UPDATED_DATE': DateTime,
}

# Function to read existing data from the database
def read_existing_data(engine):
    try:
        query = "SELECT * FROM work.tb_orderprocess_effectiveness_forecast order by forecast_date asc"
        existing_data = pd.read_sql(query, con=engine)
        
        existing_data['forecast_date'] = pd.to_datetime(existing_data['forecast_date'])
        return existing_data
    except Exception as e:
        print(f"An error occurred: {e}")

# Function to insert new data
def insert_new_data(new_data, existing_data, engine):
    # Find new rows by excluding those with existing FORECAST_DATE
    new_rows = new_data[~new_data['FORECAST_DATE'].isin(existing_data['FORECAST_DATE'])]
    
    if not new_rows.empty:
        new_rows.to_sql('tb_orderprocess_effectiveness_forecast', schema='work', con=engine, if_exists='append', index=False)
        print("New rows added successfully")
    else:
        print("No new rows to add")

try:
    # Read new data from CSV
    data = pd.read_csv("OIF_Final_Model\Future_test.csv")
    data['FORECAST_DATE'] = pd.to_datetime(data['FORECAST_DATE'])
    data['CREATED_DATE'] = pd.to_datetime(data['CREATED_DATE'])
    data['UPDATED_DATE'] = pd.to_datetime(data['UPDATED_DATE'])
    
    # Read existing data from the database
    existing_data = read_existing_data(engine)
    print(data)
    # # Insert new data if it doesn't already exist
    # insert_new_data(data, existing_data, engine)
    
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    engine.dispose()
    print("Connection closed")


C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_26120\4087280291.py:58: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['FORECAST_DATE'] = pd.to_datetime(data['FORECAST_DATE'])
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_26120\4087280291.py:59: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['CREATED_DATE'] = pd.to_datetime(data['CREATED_DATE'])
C:\Users\BharathKumar_Bellam\AppData\Local\Temp\ipykernel_26120\4087280291.py:60: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['UPDATED_DATE'] = pd.to_datetime(data['UPDATED_DATE'])


  REGION       FORECAST_DATE  MODEL1  MODEL2  MODEL3  MODEL4  MODEL5  \
0    DAO 2024-07-25 05:00:00     249     261     261     283     NaN   

   ACTUALORDERUNIT CREATED_DATE UPDATED_DATE  
0              NaN   2024-07-25   2024-07-25  
Connection closed


In [20]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.types import String, DateTime, Numeric
import cx_Oracle
from datetime import datetime as dt
import os

#SCRE
username = 'svcGOCUI'
password = 'DELL2023support#'
host = 'gocplorlvpr18.amer.dell.com'
port = '1521'
service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'

# Create DSN
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)

# Create engine
engine = create_engine(f'oracle+cx_oracle://{username}:{password}@{dsn}')
time = dt.now()
time = time.strftime('%d%m%YT%H%M')

dtype = {
    'REGION': String(256), 
    'FORECAST_DATE': DateTime,
    'MODEL1': Numeric,
    'MODEL2': Numeric,
    'MODEL3': Numeric,
    'MODEL4': Numeric,
    'MODEL5': Numeric,
    'ACTUALORDERUNIT': Numeric,
    'CREATED_DATE': DateTime,
    'UPDATED_DATE': DateTime,
}




# Function to read existing data from the database
def read_existing_data(engine):
    try:
        
        query = "SELECT * FROM work.tb_orderprocess_effectiveness_forecast order by forecast_date asc"
        existing_data = pd.read_sql(query, con=engine)
        existing_data['forecast_date'] = pd.to_datetime(existing_data['forecast_date'])
        
        # filename= f'SCREDB_BackUp_{time}.xlsx'
        # filename =os.path.join(r"C:\Users\BharathKumar_Bellam\Python for DS-ML\BackUp",filename)
        
        # existing_data.to_excel(filename,index=False)
        return existing_data
    except Exception as e:
        print(f"An error occurred: {e}")


def update_existing_rows(new_data, engine):
    try:
        with engine.connect() as conn:
            for index, row in new_data.iterrows():
                # print(index, row)
                update_query = text("""
                    UPDATE work.tb_orderprocess_effectiveness_forecast
                    SET MODEL1 = :model1, 
                        MODEL2 = :model2, 
                        MODEL3 = :model3, 
                        MODEL4 = :model4,
                        
                        UPDATED_DATE =:UPDATED_DATE
                    WHERE FORECAST_DATE = :forecast_date
                """)
                conn.execute(update_query, {
                    'model1': row['MODEL1'],
                    'model2': row['MODEL2'],
                    'model3': row['MODEL3'],
                    'model4': row['MODEL4'],
                    'UPDATED_DATE':row['UPDATED_DATE'],
                    'forecast_date': row['FORECAST_DATE']
                })
            conn.commit()
            print("existing_rows updated successfully")
    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Oracle error code: {error.code}")
        print(f"Oracle error message: {error.message}")
        
    except Exception as e:
        print(f"An error occurred: {e}")


def insert_new_data(new_data, existing_data, engine):
    try:
        # Find new rows by excluding those with existing FORECAST_DATE
        new_rows = new_data[~new_data['FORECAST_DATE'].isin(existing_data['forecast_date'])]
        
        if not new_rows.empty:
            new_rows.to_sql('tb_orderprocess_effectiveness_forecast', schema='work', con=engine, if_exists='append', index=False)
            print("New rows added successfully")
        else:
            print("No new rows to add")
    except Exception as e:
        print(f"An error occurred: {e}")

try:

    data = pd.read_csv(r"C:\Users\BharathKumar_Bellam\Python for DS-ML\Py-DS-ML-Bootcamp-master\Refactored_Py_DS_ML_Bootcamp-master\OIF_Final_Model\DataToBeImported-210824.csv")
    # print("exceldata")
    print(data)
    
    data['FORECAST_DATE'] = pd.to_datetime(data['FORECAST_DATE'])
    data['CREATED_DATE'] = pd.to_datetime(data['CREATED_DATE'])
    data['UPDATED_DATE'] = pd.to_datetime(data['UPDATED_DATE'])
    existing_data = read_existing_data(engine)
    
    existing_rows = data[data['FORECAST_DATE'].isin(existing_data['forecast_date'])]
    new_rows = data[~data['FORECAST_DATE'].isin(existing_data['forecast_date'])]
    
    print('existing_rows')
    print(existing_rows)
    print('new_rows')
    print(new_rows)
    
    if not existing_rows.empty:
        update_existing_rows(existing_rows, engine)

    if not new_rows.empty:
        insert_new_data(new_rows, existing_data, engine)
    
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    engine.dispose()
    print("Connection closed")


    REGION    FORECAST_DATE  MODEL1  MODEL2  MODEL3  MODEL4  MODEL5  \
0     EMEA   8/21/2024 5:00      56      56      70     162     NaN   
1     EMEA   8/21/2024 6:00     149     150     169     230     NaN   
2     EMEA   8/21/2024 7:00     365     368     357     304     NaN   
3     EMEA   8/21/2024 8:00     544     569     591     381     NaN   
4     EMEA   8/21/2024 9:00     786     781     810     455     NaN   
..     ...              ...     ...     ...     ...     ...     ...   
493    APJ  8/27/2024 22:00      26      27      48      35     NaN   
494    APJ  8/27/2024 23:00      61      60      86      52     NaN   
495    APJ   8/28/2024 0:00     260     270     260     398     NaN   
496    APJ   8/28/2024 1:00     420     425     557     441     NaN   
497    APJ   8/28/2024 2:00     558     561     699     479     NaN   

     ACTUALORDERUNIT CREATED_DATE UPDATED_DATE  
0                NaN    8/21/2024    8/21/2024  
1                NaN    8/21/2024    8/21/2024  
